# Model predictive speed and steering control

# MPC modeling

# Vehicle model linearization

State vector is:
$$ x = [x, y, v,\phi]$$ x: x-position, y:y-position, v:velocity, φ: yaw angle

Input vector is:
$$ u = [a, \delta]$$ a: accellation, δ: steering angle

Vehicle model is 
$$ \dot{x} = vcos(\phi)$$
$$ \dot{y} = vsin((\phi)$$
$$ \dot{v} = a$$
$$ \dot{\phi} = \frac{vtan(\delta)}{L}$$

# Reference

- [Vehicle Dynamics and Control \| Rajesh Rajamani \| Springer](http://www.springer.com/us/book/9781461414322)

- [MPC Course Material \- MPC Lab @ UC\-Berkeley](http://www.mpc.berkeley.edu/mpc-course-material)
